# 工具函数

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

D:\anaconda3\envs\extract-dialogue\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 加载模型

In [2]:
def load_model(model_dir, device='cuda'):
    """
    加载模型和分词器
    :param model_dir: 模型所在位置
    :param device: 训练设备
    :return: model，tokenizer
    """

    model = AutoModelForCausalLM.from_pretrained(
        model_dir,
        torch_dtype="auto",
        trust_remote_code=True,
    )
    model = model.to(device)

    tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

    return model, tokenizer

## 预测函数

In [3]:

def predict(model, tokenizer, prompt, device='cuda', debug=True):
    """
    文本推理
    :param model: 模型
    :param tokenizer: 分词器
    :param prompt: 提示词
    :param device: cuda or cpu
    :param debug: 是否调试
    """

    message = [{
        "role": "system",
        "content": "you are a helpful assistant."
        },
        {
            "role": "user",
            "content": prompt
        }]

    # 使用 apply_chat_template 返回文本格式的提示词
    text = tokenizer.apply_chat_template(message, tokenizer=True, add_generation_prompt=True)

    if debug: print(f"input: {text}")

    # 根据返回的类型判断如何构造 model_inputs
    if isinstance(text, list) and len(text) > 0 and isinstance(text[0], int):
        # 如果 text 是 token id 列表，直接转换为 tensor
        model_inputs = {"input_ids": torch.tensor([text]).to(device)}
    elif isinstance(text, str):
        # 如果 text 是字符串，则进行编码
        model_inputs = tokenizer([text], return_tensors="pt")
        model_inputs = {k: v.to(device) for k, v in model_inputs.items()}
    elif isinstance(text, list) and all(isinstance(x, str) for x in text):
        # 如果 text 是字符串列表
        model_inputs = tokenizer(text, return_tensors="pt")
        model_inputs = {k: v.to(device) for k, v in model_inputs.items()}
    else:
        raise ValueError("Unsupported type for text generated by apply_chat_template")

    print(f"input_ids:{model_inputs}") if debug else None

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512,
        attention_mask = model_inputs['attention_mask'],
        pad_token_id = tokenizer.pad_token_id
    )
    print(f"generated_ids:{generated_ids}") if debug else None

    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in
                     zip(model_inputs["input_ids"], generated_ids)]

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


# 测试模型能力

In [12]:
%%time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# model_dir = r'D:\Pretrained_models\Qwen\Qwen2-7B-Instruct'
model_dir = r'D:\Pretrained_models\Qwen\Qwen2-1___5B-Instruct'
model, tokenizer = load_model(model_dir=model_dir, device=device)
# for name,param in model.named_parameters():
#     print(name, param.device)
print(tokenizer.special_tokens_map)


prompt = "请简短介绍下大语言模型"
# prompt = "下面是一段对话文本，请分析对话内容是否有诈骗风险，只以json格式输出你的判断结果(is_fraud:true/false)，\n\n诈骗分子：您好，我是国家公益抽奖中心的工作人员，恭喜您获得大奖！请提供您的银行账号信息。\n用户：真的吗？我没有参与过抽奖活动，能详细说明吗？\n诈骗分子：这是随机抽取的奖项，请提供身份证号码和开户行信息以核实身份，奖金核实后立即到账。\n用户：听起来不靠谱，我需要核实，能给我官方网站或联系电话吗？\n诈骗分子：此活动为内部通知，无公开电话官网，请尽快提供信息，奖金马上到账！\n\n诈骗分子：您好，我是某某银行安全客服。检测到您的账户存在异常登录风险，为保障资金安全，请确认账户信息并修改密码。\n用户：账户异常？我没收到短信或邮件，这是什么情况？\n诈骗分子：系统升级导致部分用户需验证，请提供银行卡号和预留手机号码，我们将安排紧急冻结和风险排查。\n用户：银行通常不会这样联系，我需要去柜台核实。\n诈骗分子：为保障您的资金安全，请理解系统自动风险提示，若延误核实可能资金受损，请尽快回复信息。\n\n诈骗分子：您好，我这里有一项绝佳投资机会，保证高收益且风险极低，只需小额投资即可获得翻倍回报，您有兴趣吗？\n用户：高收益往往伴随高风险，这项目有官方认证或第三方评估报告吗？\n诈骗分子：项目由内部独家渠道获得，现处于限时开放阶段，信息保密，请先转入“风险保证金”，后续安排专业顾问跟进。\n用户：风险太大，我不想冒险，请先提供更多证明材料。\n诈骗分子：机会稍纵即逝，投资越早回报越快，请您立即转账，我将发送详细后续流程说明。"
result = predict(model=model, tokenizer=tokenizer, prompt=prompt, device=device, debug=False)
print(result)

{'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}
大语言模型是一种人工智能技术，它能够理解、生成和处理自然语言。它们可以用于各种任务，包括文本生成、问答系统、自动翻译等。这些模型通常由深度学习算法驱动，并且可以根据输入的语句进行训练和调整。
CPU times: total: 656 ms
Wall time: 3.26 s


In [11]:
print(model.generate.__doc__)



        Generates sequences of token ids for models with a language modeling head.

        <Tip warning={true}>

        Most generation-controlling parameters are set in `generation_config` which, if not passed, will be set to the
        model's default generation configuration. You can override any `generation_config` by passing the corresponding
        parameters to generate(), e.g. `.generate(inputs, num_beams=4, do_sample=True)`.

        For an overview of generation strategies and code examples, check out the [following
        guide](../generation_strategies).

        </Tip>

        Parameters:
            inputs (`torch.Tensor` of varying shape depending on the modality, *optional*):
                The sequence used as a prompt for the generation or as model inputs to the encoder. If `None` the
                method initializes it with `bos_token_id` and a batch size of 1. For decoder-only models `inputs`
                should be in the format of `input_ids`. For enc